In [25]:
import pandas as pd
import sqlite3

## Creating a connection to the database using the library sqlite3

In [26]:
db_con = sqlite3.connect('../data/checking-logs.sqlite.sqlite')

## Get the schema of the table test

In [27]:
query = "PRAGMA table_info(checker);"
pd.io.sql.read_sql(query, db_con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## Get only the first 10 rows of the table test to check what the table looks like

In [28]:
query = "SELECT * FROM test LIMIT 10"
pd.io.sql.read_sql(query, db_con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
- do this by joining the table with the table deadlines
- the difference should be displayed in hours
- do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier
- the value should be stored in the dataframe df_min with the corresponding uid

In [29]:
query = "SELECT test.uid, MIN(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_min = pd.io.sql.read_sql(query, db_con)
df_min

,uid,dif_time
0,user_30,-202


## Do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [30]:
query = "SELECT test.uid, MAX(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_max = pd.io.sql.read_sql(query, db_con)
df_max

,uid,dif_time
0,user_25,-2


## Do the same thing but for the average,using only one query,this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [31]:
query = "SELECT AVG(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_avg = pd.io.sql.read_sql(query, db_con)
df_avg

,dif_time
0,-89.125


## test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [32]:
query = '''SELECT test.uid as uid,
AVG(CAST((julianday(test.first_commit_ts) -
      julianday(datetime(deadlines.deadlines, 'unixepoch')))
     * 24 AS integer))
AS avg_diff, views.av AS pageviews
FROM test
JOIN deadlines ON test.labname=deadlines.labs
JOIN (SELECT uid, COUNT(uid) AS av
      from pageviews GROUP By uid)
      AS views ON test.uid=views.uid
WHERE labname<>'project1'
GROUP BY test.uid
'''
views_diff = pd.io.sql.read_sql(query, db_con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [33]:
views_diff.corr()

/var/folders/zz/zyxvpxvq6csfxvn_n0001wx0000g78/T/ipykernel_53814/1431885951.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  views_diff.corr()


,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## Close the connection

In [34]:
db_con.close()